pretrain affine Q network

In [3]:
# versioninfo()
Base.active_project()


"/home/jiaxingl/project/Cersyve.jl/Project.toml"

In [6]:
using Pkg
using Revise

Pkg.activate("/home/jiaxingl/project/verify_julia_env")
Pkg.status()


  Activating project at `~/project/verify_julia_env`


Status `~/project/verify_julia_env/Project.toml`
  [d8c2afa5] Cersyve v1.0.0-DEV `~/project/Cersyve.jl`
⌅ [587475ba] Flux v0.13.17
  [f67ccb44] HDF5 v0.17.2
  [7073ff75] IJulia v1.26.0
⌅ [033835bb] JLD2 v0.4.53
⌅ [b4f0291d] LazySets v1.59.1
  [6d061d49] ModelVerification v0.1.0 `/home/jiaxingl/project/ModelVerification.jl#cersyve`
  [85610aed] NaiveNASflux v2.0.8 `~/project/ModelVerification.jl/onnx_parser/NaiveNASflux`
⌃ [bd45eb3e] NaiveNASlib v2.0.11
⌃ [d0dd6a25] ONNX v0.2.0
  [2e935253] ONNXNaiveNASflux v0.2.7 `~/project/ModelVerification.jl/onnx_parser/ONNXNaiveNASflux`
⌅ [3bd65402] Optimisers v0.2.20
  [49802e3a] ProgressBars v1.5.1
  [438e738f] PyCall v1.96.4
  [295af30f] Revise v3.6.4
⌅ [2913bbd2] StatsBase v0.33.21
⌃ [899adc3e] TensorBoardLogger v0.1.19
  [e88e6eb3] Zygote v0.6.73
  [37e2e46d] LinearAlgebra
  [56ddb016] Logging
  [de0858da] Printf
  [9a3f8284] Random
  [10745b16] Statistics v1.9.0
Info Packages marked with ⌃ and ⌅ have new versions available, but those with ⌅ a

In [11]:
# # project_path = "/home/jiaxingl/project/verify_julia_env/Project.toml"
# # open(project_path) do file
# #     println(read(file, String))
# # end

# manifest_path = "/home/jiaxingl/project/verify_julia_env/Manifest.toml"
# open(manifest_path) do file
#     println(read(file, String))
# end

In [7]:
using Revise
using Cersyve
using Flux
using JLD2
using Random

function create_filter_matrix(start_idx, end_idx, total_len)
    W = zeros(end_idx - start_idx + 1, total_len)
    for i in start_idx:end_idx
        # println(i - start_idx + 1)
        W[i - start_idx + 1, i] = 1.0
    end
    return W
end

function create_expand_matrix(start_idx, end_idx, expand_dim)
    input_dim = end_idx - start_idx + 1

    W = zeros(expand_dim, input_dim)
    for i in start_idx:end_idx
        # println(i - start_idx + 1)
        W[i, i - start_idx + 1] = 1.0
    end
    return W
end

function create_parallel_affine_Q(x_dim, u_dim)
    # can take both x and xu as input
    # function filter_x(input)
    #     return input[1:x_dim, :]
    # end
    
    # function filter_u(input)
    #     return input[x_dim+1:end, :]
    # end
    W_x = create_filter_matrix(1, x_dim, x_dim + u_dim)
    b_x = zeros(x_dim)
    filter_x = Dense(W_x, b_x)
    W_u = create_filter_matrix(x_dim + 1, x_dim + u_dim, x_dim + u_dim)
    u_b = zeros(u_dim)
    filter_u = Dense(W_u, u_b)

    # start_idx, end_idx, expand_dim
    b1_embedding_dim = 32
    W_expand_emb = create_expand_matrix(1, b1_embedding_dim, b1_embedding_dim + u_dim)
    b_expand_emb = zeros(b1_embedding_dim + u_dim)
    expand_emb = Dense(W_expand_emb, b_expand_emb)

    W_expand_u = create_expand_matrix(b1_embedding_dim + 1, b1_embedding_dim + u_dim, b1_embedding_dim + u_dim)
    b_expand_u = zeros(b1_embedding_dim + u_dim)
    expand_u = Dense(W_expand_u, b_expand_u)

    # Branch 1
    b1 = Chain(
        filter_x,
        Dense(x_dim, 32, relu),
        Dense(32, 32, relu),
        Dense(32, 1, relu)
    )
    b2 = Chain(
        filter_u,
        Dense(u_dim, 1)
    )
    # final_layer = Chain(Dense(32 + u_dim, 1))

    # Complete model
    model = Chain(
        Parallel(
            +, 
            b1,
            b2
        ),
        # final_layer  
    )
    return model
end



task = Unicycle
value_hidden_sizes = [32, 32]
dynamics_hidden_sizes = [32, 32]
constraint_hidden_sizes = [16]
data_path = joinpath(@__DIR__, "../data/unicycle_data.jld2")
model_dir = joinpath(@__DIR__, "../model/unicycle/")
log_dir = joinpath(@__DIR__, "../log/unicycle/")
seed = 1

Random.seed!(seed)

# Q_model = Cersyve.create_mlp(task.x_dim, 1, value_hidden_sizes)
# Q_model = Cersyve.create_mlp(task.x_dim + task.u_dim, 1, value_hidden_sizes)


data = JLD2.load(data_path)["data"]
f_model = Cersyve.create_mlp(task.x_dim + task.u_dim, task.x_dim, dynamics_hidden_sizes)
Flux.loadmodel!(f_model, JLD2.load(joinpath(model_dir, "f.jld2"), "state"))
f_pi_model = Cersyve.create_closed_loop_dynamics_model(
    f_model, task.pi_model, data, task.x_low, task.x_high, task.u_dim)

h_model = Cersyve.create_mlp(task.x_dim, 1, constraint_hidden_sizes)
Flux.loadmodel!(h_model, JLD2.load(joinpath(model_dir, "h.jld2"), "state"))

x_a_low =  [task.x_low; task.u_low]
x_a_high = [task.x_high; task.u_high]


affine_Q = create_parallel_affine_Q(task.x_dim, task.u_dim)

# trainable parameters
# println(affine_Q[1][1][2])
# println(affine_Q[1][1][3])
# println(affine_Q[2])

# pretrain_Q(
#     affine_Q,
#     f_pi_model,
#     task.pi_model,
#     h_model,
#     task.x_low,
#     task.x_high;
#     penalty="APA",
#     space_size=x_a_high - x_a_low,
#     apa_coef=1e-4,
#     log_dir=log_dir,
# )




Chain(
  Parallel(
    +,
    Chain(
      Dense(5 => 3),                    # 18 parameters
      Dense(3 => 32, relu),             # 128 parameters
      Dense(32 => 32, relu),            # 1_056 parameters
      Dense(32 => 1, relu),             # 33 parameters
    ),
    Chain(
      Dense(5 => 2),                    # 12 parameters
      Dense(2 => 1),                    # 3 parameters
    ),
  ),
)                   # Total: 12 arrays, 1_250 parameters, 6.125 KiB.

In [25]:
Q_prime, Q_interval = create_Q_Q_prime(affine_Q, f_pi_model, task)

base_input = zeros(task.x_dim + task.u_dim)
# println(Q_interval[1].layers[1](base_input))

# println(Q_interval[1].layers[2](base_input))
println(affine_Q[1].layers[2].layers[2].weight)
println(affine_Q[1].layers[2].layers[2].bias)
# println(Q_interval[1].layers[2].layers[3].weight)
# println(Q_interval[1].layers[3].layers[3].weight)

# output = Q_interval[1].layers[2].layers[1](base_input) 
# println(output)
# output = Q_interval[1].layers[2].layers[2](output)
# println(output)
# output = Q_interval[1].layers[2].layers[3](output)
# println(output)
# println(Q_interval[1].layers[2].layers[1](base_input))
# println(Q_interval[1].layers[3](base_input))

println(affine_Q(base_input))
println(Q_interval(base_input))

PASS 0
PASS 1
PASS 2
Float32[-0.19020113 0.08185339]
Float32[0.0]
Float32[0.0]
Float32[-0.27205452]


In [31]:
task = Unicycle
value_hidden_sizes = [32, 32]
dynamics_hidden_sizes = [32, 32]
constraint_hidden_sizes = [16]
data_path = joinpath(@__DIR__, "../data/unicycle_data.jld2")
model_dir = joinpath(@__DIR__, "../model/unicycle/")
log_dir = joinpath(@__DIR__, "../log/unicycle/")
seed = 1

Random.seed!(seed)


data = JLD2.load(data_path)["data"]
f_model = Cersyve.create_mlp(task.x_dim + task.u_dim, task.x_dim, dynamics_hidden_sizes)
Flux.loadmodel!(f_model, JLD2.load(joinpath(model_dir, "f.jld2"), "state"))
f_pi_model = Cersyve.create_closed_loop_dynamics_model(
    f_model, task.pi_model, data, task.x_low, task.x_high, task.u_dim)

h_model = Cersyve.create_mlp(task.x_dim, 1, constraint_hidden_sizes)
Flux.loadmodel!(h_model, JLD2.load(joinpath(model_dir, "h.jld2"), "state"))

x_a_low =  [task.x_low; task.u_low]
x_a_high = [task.x_high; task.u_high]

# fun_affine_Q = create_func_parallel_affine_Q(task.x_dim, task.u_dim)
affine_Q = create_parallel_affine_Q(task.x_dim, task.u_dim)
Flux.loadmodel!(affine_Q, JLD2.load(joinpath(model_dir, "Q_pretrain.jld2"), "state"))

finetune_Q(
    task, 
    affine_Q,
    f_pi_model,
    h_model,
    x_a_low,
    x_a_high;
    log_dir=log_dir,
)


PASS 0
PASS 1
PASS 2


0.0%┣                                           ┫ 0/100.0k [00:00<00:-1, -0s/it]
0.0%┣                                       ┫ 1/100.0k [00:06<Inf:Inf, InfGs/it]
0.0%┣                                         ┫ 3/100.0k [00:07<96:53:16, 3s/it]
0.0%┣                                         ┫ 4/100.0k [00:07<66:05:46, 2s/it]
0.0%┣                                        ┫ 10/100.0k [00:07<22:20:45, 1it/s]
0.0%┣                                        ┫ 16/100.0k [00:07<13:39:37, 2it/s]
0.0%┣                                        ┫ 22/100.0k [00:07<09:49:31, 3it/s]
0.0%┣                                        ┫ 25/100.0k [00:07<08:39:52, 3it/s]
0.0%┣                                        ┫ 29/100.0k [00:08<07:31:18, 4it/s]
0.0%┣                                        ┫ 32/100.0k [00:08<06:54:15, 4it/s]
0.0%┣                                        ┫ 37/100.0k [00:08<05:59:58, 5it/s]
0.0%┣                                        ┫ 44/100.0k [00:08<05:03:30, 5it/s]
0.0%┣                       

----- Verification Starts -----


LoadError: Gurobi Error 10009: No Gurobi license found (user jiaxingl, host icl-superman, hostid efaed98b, cores 24)

Testing Q_Q_next model where Q_next is calculating argmin_u (Q) with interval arithmatics

In [ ]:
struct ExpandXToXU
    x_dim::Int
    u_dim::Int
end

# Define the forward pass for the layer
function (layer::ExpandXToXU)(x)
    # `x` is the input vector of size `x_dim`
    u = rand(layer.u_dim)  # Generate random `u` part
    return vcat(x, u)      # Concatenate `x` and `u`
end

# Example usage
function create_expand_xu_layer(x_dim, u_dim)
    return ExpandXToXU(x_dim, u_dim)
end

# Custom Dense Layer for Interval Arithmetic
struct DenseInterval
    W::AbstractMatrix
    b::AbstractVector
    x_dim::Int
    u_low::AbstractVector
    u_up::AbstractVector
end

function DenseInterval(W, b, x_dim, u_low, u_up)
    DenseInterval(W, b, x_dim, u_low, u_up)
end

# Forward pass for interval arithmetic
function (layer::DenseInterval)(xu::AbstractVector)
    # Compute lower and upper bounds for each neuron
    x_dim = size(xu)[1] - size(layer.u_low)[1]
    x = xu[1:x_dim, :]

    W_x = layer.W[:, 1:x_dim]
    z = W_x * x + layer.b
    println(z)

    W_u = layer.W[:, x_dim+1:end]
    W_up = max.(W_u, 0.0)
    W_un = min.(W_u, 0.0)
    println(W_u)

    l = W_up * layer.u_low .+ W_un * layer.u_up .+ z
    return l
end

# Create the model
function create_parallel_affine_Q_interval(x_dim, u_dim, u_low, u_up)

    x_w = create_filter_matrix(1, x_dim, x_dim+u_dim)
    u_w = create_filter_matrix(x_dim, x_dim+u_dim-1, x_dim+u_dim)
    
    function filter_x(input)
        return input[1:x_dim]
    end
    
    function filter_u(input)
        return input[x_dim+1:end]
    end
    
    # Branch 1
    b1 = Chain(
        filter_x,
        Dense(x_dim, 32, relu),
        Dense(32, 32, relu)
    )

    # Define the interval arithmetic layer for the final output
    final_layer_interval = DenseInterval(randn(1, 32 + u_dim), randn(1), task.x_dim, task.u_low, task.u_high)

    # Complete model
    model = Chain(
        Parallel(
            vcat, 
            b1,
            filter_u
        ),
        final_layer_interval  # Perform interval arithmetic here
    )
    return model
end
function create_Q_constraint_model(Q_model, h_model)
    x_w = create_filter_matrix(1, task.x_dim, task.x_dim + task.u_dim)
    x_b = zeros(task.x_dim)
    filter_x = Dense(x_w, x_b)
    return Chain(Parallel(+,
        Chain(filter_x, h_model, Dense(Float32[1; 0;;])),
        Chain(affine_Q,  Dense(Float32[0; 1;;]))
    ))
end


function create_Q_Q_prime(affine_Q, task)
    # trainable parameters
    # println(affine_Q[1][1][2])
    # println(affine_Q[1][1][3])
    # println(affine_Q[2])
    
    # creating Q_prime
    affine_Q_interval = create_parallel_affine_Q_interval(task.x_dim, task.u_dim, task.u_low, task.u_high)
    println(affine_Q_interval)
    # Copy weights and biases from affine_Q to affine_Q_interval
    affine_Q_interval[1].layers[1].layers[2].weight .= affine_Q[1].layers[1].layers[2].weight
    affine_Q_interval[1].layers[1].layers[2].bias .= affine_Q[1].layers[1].layers[2].bias
    affine_Q_interval[1].layers[1].layers[3].weight .= affine_Q[1].layers[1].layers[3].weight
    affine_Q_interval[1].layers[1].layers[3].bias .= affine_Q[1].layers[1].layers[3].bias
    println("PASS 0")
    # Map the final layer to DenseInterval
    affine_Q_interval[2].W .= affine_Q[2].layers[1].weight
    affine_Q_interval[2].b .= affine_Q[2].layers[1].bias
    println("PASS 1")
    x_w = create_filter_matrix(1, task.x_dim, task.x_dim + task.u_dim)
    x_b = zeros(task.x_dim)
    function filter_x(input)
        return input[1:task.x_dim]
    end
    expand_layer = create_expand_xu_layer(task.x_dim, task.u_dim)
    println("PASS 2")
    return Chain(Parallel(+,
        Chain(affine_Q, Dense(Float32[1; 0;;])),
        Chain(filter_x, f_pi_model, expand_layer, affine_Q_interval, Dense(Float32[0; 1;;])),
    ))
end




Q_Q_prime = create_Q_Q_prime(affine_Q, task)
# println(Q_Q_prime)


# Q_interval = create_parallel_affine_Q_interval(2, 2, [-1, -1], [1, 1])
# input = Float32.(rand(4))
# Q_interval(input)
# Q_Q_prime(rand(task.x_dim + task.u_dim))


Chain(Parallel(vcat, Chain(filter_x, Dense(3 => 32, relu), Dense(32 => 32, relu)), filter_u), DenseInterval([1.309654063351668 -0.6390056940444813 0.7496010905967507 0.20168524622744236 -1.400640726825294 -0.4148314861667896 0.24807933728887804 0.3276641996329063 1.1075522392638797 1.3831249033747295 -0.6946088766507229 -0.31262456505478 -0.3454788751499618 2.288054426572957 -1.0674940294687225 -0.32933759031692106 -1.5649916867785374 0.880395799308013 -0.05879943024045937 2.1039865719748794 -0.8445200215144919 0.4451827137673766 1.4465613536949224 0.8860690717585733 -1.1921019894712768 0.9187515789292563 -0.30612467691064815 0.8543222806454905 -0.778849112543019 -2.333542036806978 -1.7467994990489675 0.43362032531006894 -0.9554661679083543 1.8237798977377517], [1.8662738837122657], 3, Float32[-1.0, -1.0], Float32[1.0, 1.0]))
PASS 0
PASS 1
PASS 2
[-0.5119603069150069;;]
[0.049090515822172165 0.08699902147054672]


2×1 Matrix{Float32}:
 -0.46235076
 -0.64804983

Test affine Q BGB


In [29]:
function find_min_at_vertices(Q_model, x::Vector{Float32}, u_low::Vector{Float32}, u_high::Vector{Float32}, x_dim::Int)
    # Step 1: Compute vertices
    vertices = compute_vertices(u_low, u_high)  # Each column is a vertex

    # Step 2: Evaluate Q_model at each vertex
    min_value = Inf
    min_vertex = nothing
    for i in 1:size(vertices, 2)
        u = vertices[:, i]             # Extract the i-th vertex (u part)
        xu = copy(x)                   # Clone x
        xu[x_dim+1:end] .= u           # Replace u portion in x
        Q_val = Q_model(xu)[1]         # Evaluate Q_model (assume scalar output)
        # Step 3: Check if this is the minimum
        if Q_val < min_value
            min_value = Q_val
            min_vertex = u
        end
    end

    return min_value
end


function compute_vertices(u_low::Vector{Float32}, u_high::Vector{Float32})
    n = length(u_low)  # Dimension of the space
    vertices = []      # Initialize an empty array to store vertices

    # Iterate over all 2^n combinations
    for i in 0:(2^n - 1)
        vertex = Float32[]  # Initialize a vertex
        for j in 1:n
            # Check the j-th bit of i to decide low or high
            if (i >> (j - 1)) & 1 == 0
                push!(vertex, u_low[j])
            else
                push!(vertex, u_high[j])
            end
        end
        push!(vertices, vertex)  # Add the vertex to the list
    end

    return reduce(hcat, vertices)  # Return vertices as a matrix (each column is a vertex)
end

function boundary_guided_search_Q(
    task::Any,
    x::Matrix{Float32},
    x_low::Vector{Float32},
    x_high::Vector{Float32},
    h_model::Any,
    Q_model::Any,
    f_pi_model::Any;
    pgd_step::Int64 = 10,
    pgd_eps::Float64 = 0.1,
    pgd_beta::Float64 = 0.0,
    backtrack_step::Int64 = 20,
    length_discount::Float64 = 0.8,
    bound_guide::Bool = true,
    direct_discount::Float64 = 0.5,
    tol::Float64 = 1e-4,
)::Matrix{Float32}
"""
1. change the input of h_model and f_pi_model to x[:task.x_dim, :]
2. change the properties of verification, but v and v_prime are maintained, represent q and q_prime now
"""
    pgd = ones(Bool, size(x, 2))
    x_pgd = x
    m = zeros(Float32, size(x))

    for _ in 1:pgd_step
        h = h_model(x_pgd[:task.x_dim, :])[1, :]
        v = Q_model(x_pgd)[1, :]
        # v_prime = Q_model(f_pi_model(x_pgd[:task.x_dim,:]))[1, :]
        v_prime = find_min_at_vertices(Q_model, x_pgd, task.u_low, task.u_high, task.x_dim)
        
        con = (v .<= tol) .& (h .> -tol)
        inv = (v .<= tol) .& (v_prime .> -tol)
        
        pgd_pgd = pgd[pgd]
        pgd_pgd[con .| inv] .= 0
        pgd[pgd] = pgd_pgd
        x_pgd = x[:, pgd]

        con_g = Flux.gradient(x -> sum(h_model(x[:task.x_dim, :])), x_pgd[:, 1:div(size(x_pgd, 2), 2)])[1]
        inv_g = Flux.gradient(x -> sum(Q_model(f_pi_model(x[:task.x_dim, :]))), x_pgd[:, size(con_g, 2) + 1:end])[1]
        g = hcat(con_g, inv_g) + Float32(pgd_beta) * m[:, pgd]
        g ./= sqrt.(sum(g .^ 2, dims=1))

        v_g = Flux.gradient(x -> sum(Q_model(x)), x_pgd)[1]
        v_g ./= sqrt.(sum(v_g .^ 2, dims=1))

        a = sum(g .* v_g, dims=1)
        z = a .* g - v_g
        z ./= sqrt.(sum(z .^ 2, dims=1))

        dirc = zeros(Float32, size(x_pgd))
        coef = zeros(Float32, size(x_pgd, 2))
        tmp = ones(Bool, size(x_pgd, 2))

        for i in 1:backtrack_step + 1
            if bound_guide
                d_coef = Float32(direct_discount ^ (i - 1))
                d = d_coef * g[:, tmp] + (1 - d_coef) * z[:, tmp]
                d ./= sqrt.(sum(d .^ 2, dims=1))
            else
                d = g[:, tmp]
            end

            l_coef = Float32(length_discount ^ (i - 1))
            x_tmp = x_pgd[:, tmp] + Float32.(l_coef * pgd_eps) * d
            x_tmp = min.(max.(x_tmp, x_low), x_high)

            fea = Q_model(x_tmp)[1, :] .<= tol

            dirc[:, tmp] = d

            coef_tmp = coef[tmp]
            coef_tmp[fea] .= l_coef
            coef[tmp] = coef_tmp

            tmp_tmp = tmp[tmp]
            tmp_tmp[fea] .= 0
            tmp[tmp] = tmp_tmp

            if maximum(tmp; init=0) == 0
                break
            end
        end
        dx = reshape(coef, 1, size(x_pgd, 2)) .* dirc
        x_pgd = x_pgd + Float32(pgd_eps) * dx
        x_pgd = min.(max.(x_pgd, x_low), x_high)
        x[:, pgd] = x_pgd

        m_pgd = m[:, pgd]
        m_pgd[:, .~tmp] = dx[:, .~tmp]
        m[:, pgd] = m_pgd
    end
    return x
end

function filter_counterexample_Q(
    task::Any,
    x::Matrix{Float32},
    h_model::Any,
    Q_model::Any,
    f_pi_model::Any;
    tol::Float64 = 1e-4,
)::Tuple{BitVector, BitVector}
    h = h_model(x)[1, :]
    v = Q_model(x)[1, :]
    v_prime = find_min_at_vertices(Q_model, x_pgd, task.u_low, task.u_high, task.x_dim)[1, :]
    con = (v .<= tol) .& (h .> -tol)
    inv = (v .<= tol) .& (v_prime .> -tol) .& (.~con)
    return con, inv
end

filter_counterexample_Q (generic function with 1 method)

Test Finetune affine_Q

In [30]:
function finetune_Q(
    task::Any,
    Q_model::Any,
    f_pi_model::Any,
    h_model::Any,
    x_low::Vector{Float32},
    x_high::Vector{Float32};
    lr::Float64 = 1e-4,
    max_iter::Int64 = 100000,
    search_size::Int64 = 1000,
    bnd_ratio::Float64 = 0.1,
    bnd_ratio_avg::Float64 = 0.9,
    min_bnd_ratio::Float64 = 0.01,
    max_bnd_ratio::Float64 = 1.0,
    bnd_eps::Float64 = 0.1,
    search_method::String = "BGB",  # "BGB" / "PGD-B" / "PBS"
    pgd_step::Int64 = 10,
    pgd_eps::Float64 = 0.1,
    backtrack_step::Int64 = 20,
    length_discount::Float64 = 0.8,
    direct_discount::Float64 = 0.5,
    tol::Float64 = 1e-4,
    capacity::Int64 = 10000,
    sample_size::Int64 = 100,
    search_stop::Int64 = 1000,
    replay::Int64 = 1,
    max_skip::Int64 = 1000,
    reg_method::Union{String, Nothing} = "ESR",  # "ESR" / "RSR" / nothing
    esr_max_con::Union{Int64, Nothing} = nothing,
    eps_h::Float64 = 0.01,
    eps_v::Float64 = 0.01,
    reg_coef::Float64 = 0.1,
    log_dir::Union{String, Nothing} = nothing,
    eval_every::Int64 = 10,
    save_every::Int64 = 1000,
)
    function update_value_network!(model::Any, loss_fn, opt_state, trainable_params)
        # Calculate gradients
        loss, grad = Flux.withgradient(loss_fn, model)
        value_grad = grad[1][:value_network]
        #println(typeof(value_grad))
        #println(sizeof(value_grad))
        #grad = Flux.gradient(() -> loss_fn, Flux.params(model.value_network))
        
        Flux.update!(opt_state, model.value_network, value_grad)
        return loss
    end
    

    skipped = 0
    verified = 0
    con_start_values = nothing
    inv_start_values = nothing

    buffer = Buffer(capacity, length(x_low))

    ######################################################
    trainable_params = Flux.params(Q_model[1][1][2], Q_model[1][1][3], Q_model[2])
    opt_state = Flux.setup(Adam(lr), trainable_params)
    #opt_state = Flux.setup(Adam(lr), Q_model[2])
    ######################################################
    if isnothing(log_dir)
        log_dir = joinpath(@__DIR__, "../log/")
    end
    log_path = joinpath(log_dir, "finetune_" * Dates.format(Dates.now(), "yyyymmdd_HHMMSS"))
    logger = TBLogger(log_path)

    Q_h_model = create_Q_constraint_model(Q_model, h_model)
    Q_Q_prime_model = create_Q_Q_prime(Q_model, task)

    for i in ProgressBar(1:max_iter)
        if (length(buffer.stored) < search_stop)
            x = uniform(x_low, x_high, round(Int64, search_size / bnd_ratio))
            v = Q_model(x)[1, :]
            x_bnd = x[:, (v .> -bnd_eps) .& (v .<= tol)]
            bnd_ratio = bnd_ratio_avg * bnd_ratio + (1 - bnd_ratio_avg) * size(x_bnd, 2) / size(x, 2)
            bnd_ratio = clamp(bnd_ratio, min_bnd_ratio, max_bnd_ratio)

            if search_method == "BGB"
                x_pgd = boundary_guided_search_Q(task, x_bnd, x_low, x_high, h_model, Q_model, f_pi_model;
                    pgd_step=pgd_step, pgd_eps=pgd_eps, backtrack_step=backtrack_step,
                    length_discount=length_discount, bound_guide=true, direct_discount=direct_discount,
                    tol=tol)
            end
            con, inv = filter_counterexample_Q(task, x_pgd, h_model, Q_model, f_pi_model; tol=tol)
            ce = con .| inv
            push!(buffer, x_pgd[:, ce])

            with_logger(logger) do
                @info "finetune" searched_boundary_states=size(x_bnd, 2) log_step_increment=0
                @info "finetune" boundary_state_ratio=bnd_ratio log_step_increment=0
                @info "finetune" searched_constraint_counterexample=sum(con) log_step_increment=0
                @info "finetune" searched_invariance_counterexample=sum(inv) log_step_increment=0
            end
        end

        if length(buffer.stored) > 0
            skipped = 0

            n = min(sample_size, length(buffer.stored))
            x, c = pop!(buffer, n)
            con, inv = filter_counterexample_Q(task, x, h_model, Q_model, f_pi_model; tol=tol)
            x_con, x_inv = x[:, con], x[:, inv]
            c[con .| inv] .= 0
            c[.~con .& .~inv] .+= 1
            push_idx = c .< replay
            push!(buffer, x[:, push_idx], c[push_idx])

            n_con, n_inv = size(x_con, 2), size(x_inv, 2)

            if !isnothing(reg_method) && (n_con + n_inv > 0) && (
                isnothing(esr_max_con) || (n_con + n_inv < esr_max_con))
                if reg_method == "ESR"
                    # entering state regularization
                    x_reg = uniform(x_low, x_high, search_size)
                    h_reg = h_model(x_reg[:task.x_dim, :])[1, :]
                    v_reg = Q_model(x_reg)[1, :]
                    v_reg_prime = Q_model(f_pi_model(x_reg[:task.x_dim, :]))[1, :]
                    entering = (h_reg .<= -eps_h) .& (v_reg .> 0) .& (
                        v_reg .<= eps_v) .& (v_reg_prime .<= -eps_v)
                    x_reg = x_reg[:, entering]
                    n_reg = size(x_reg, 2)
                elseif reg_method == "RSR"
                    # random state regularization
                    x_reg = uniform(x_low, x_high, search_size)
                    n_reg = size(x_reg, 2)
                end
            else
                n_reg = 0
            end

            function value_loss_fn(Q_model)
                if n_con > 0
                    con_loss = sum(-Q_model(x_con))
                else
                    con_loss = 0
                end

                if n_inv > 0
                    inv_loss = sum(-Q_model(x_inv) + Q_model(f_pi_model(x_inv[:task.x_dim, :])))
                else
                    inv_loss = 0
                end

                if n_reg > 0
                    reg_loss = mean(Q_model(x_reg))
                else
                    reg_loss = 0
                end
                # println("n_con: ", con_loss)
                # println("n_inv: ", inv_loss)
                # println("n_reg: ", reg_loss)
                loss = (con_loss + inv_loss) / max(n_con + n_inv, 1) + reg_coef * reg_loss
                return loss
            end
            
            # regular
            loss, grad = Flux.withgradient(value_loss_fn, Q_model)
            Flux.update!(opt_state, trainable_params, grad[1])
            
            ######################################################
            # finetune rcppol
            # loss, grad = Flux.withgradient(value_loss_fn, Q_model)
            # value_grad = grad[1]
            # if isnothing(value_grad)
            #     println(grad)
            #     continue
            # end
            # value_grad = value_grad[1][2]
            # Flux.update!(opt_state, Q_model[2], value_grad)
            ######################################################

            with_logger(logger) do
                @info "finetune" sample_size=n log_step_increment=0
                @info "finetune" value_loss=loss log_step_increment=0
                @info "finetune" sampled_constraint_counterexample=n_con log_step_increment=0
                @info "finetune" sampled_invariance_counterexample=n_inv log_step_increment=0
                if !isnothing(reg_method)
                    @info "finetune" regularization_state=n_reg log_step_increment=0
                end
            end
        else
            skipped += 1
        end

        if skipped == max_skip
            jldsave(joinpath(log_path, "Q_finetune.jld2"); state=Flux.state(Q_model))
            println("----- Verification Starts -----")
            con_res, inv_res = verify_value(x_low, x_high, Q_h_model, Q_Q_prime_model;
                con_start_values=con_start_values, inv_start_values=inv_start_values)
            println("----- Verification Ends -----")

            verified += 1

            if (con_res.status == :holds) & (inv_res.status == :holds)
                break
            else
                if con_res.status == :violated
                    ce = Float32.(con_res.info[:counter_example])
                    push!(buffer, reshape(ce, length(ce), 1))
                    println("Constraint counterexample: ", ce)
                    con_start_values = con_res.info[:verified_bounds][:values]
                end
                if inv_res.status == :violated
                    ce = Float32.(inv_res.info[:counter_example])
                    push!(buffer, reshape(ce, length(ce), 1))
                    println("Invariance counterexample: ", ce)
                    inv_start_values = inv_res.info[:verified_bounds][:values]
                end
                println("")
                skipped = 0
            end
        end

        with_logger(logger) do
            @info "finetune" total_counterexample=length(buffer.stored)
            @info "finetune" skipped_update=skipped log_step_increment=0
            @info "finetune" verified_times=verified log_step_increment=0
        end

        if i % eval_every == 0
            fea_rate = mean(Q_model(uniform(x_low, x_high, search_size)) .<= 0)

            with_logger(logger) do
                @info "finetune" predicted_feasible_rate=fea_rate log_step_increment=0
            end
        end

        if i % save_every == 0
            jldsave(joinpath(log_path, "V_finetune.jld2"); state=Flux.state(Q_model))
        end
    end
end


finetune_Q (generic function with 1 method)

In [ ]:
using Pkg
using Dates
# Pkg.add("TensorBoardLogger")
Pkg.add("Printf")              # Standard library, no need to add in most cases
Pkg.add("ProgressBars")
Pkg.add("Random")              # Standard library, no need to add in most cases
Pkg.add("Statistics")          # Standard library, no need to add in most cases
Pkg.add("StatsBase")
Pkg.add("TensorBoardLogger")
Pkg.add("Zygote")
Pkg.add("PyCall")
Pkg.add("HDF5")
Pkg.activate("/home/jiaxingl/project/verify_julia_env")
using Dates
using Flux
using JLD2
using LazySets
using LinearAlgebra
using Logging
using ModelVerification
using Printf
using ProgressBars
using Random
using Statistics
using StatsBase
using TensorBoardLogger
using Zygote
using PyCall
using HDF5
using TensorBoardLogger
using ProgressBars
# Pkg.status()
include("/home/jiaxingl/BoArchive/project/Cersyve.jl/src/buffer.jl")



task = Unicycle
value_hidden_sizes = [32, 32]
dynamics_hidden_sizes = [32, 32]
constraint_hidden_sizes = [16]
data_path = joinpath(@__DIR__, "../data/unicycle_data.jld2")
model_dir = joinpath(@__DIR__, "../model/unicycle/")
log_dir = joinpath(@__DIR__, "../log/unicycle/")
seed = 1

Random.seed!(seed)


data = JLD2.load(data_path)["data"]
f_model = Cersyve.create_mlp(task.x_dim + task.u_dim, task.x_dim, dynamics_hidden_sizes)
Flux.loadmodel!(f_model, JLD2.load(joinpath(model_dir, "f.jld2"), "state"))
f_pi_model = Cersyve.create_closed_loop_dynamics_model(
    f_model, task.pi_model, data, task.x_low, task.x_high, task.u_dim)

h_model = Cersyve.create_mlp(task.x_dim, 1, constraint_hidden_sizes)
Flux.loadmodel!(h_model, JLD2.load(joinpath(model_dir, "h.jld2"), "state"))

x_a_low =  [task.x_low; task.u_low]
x_a_high = [task.x_high; task.u_high]


affine_Q = create_parallel_affine_Q(task.x_dim, task.u_dim)


finetune_Q(
    task, 
    affine_Q,
    f_pi_model,
    h_model,
    x_a_low,
    x_a_high;
    log_dir=log_dir,
)



LoadError: UndefVarError: `Buffer` not defined

In [6]:
function create_filter_matrix(start_idx, end_idx, total_len)
    W = zeros(end_idx - start_idx + 1, total_len)
    for i in start_idx:end_idx
        println(i - start_idx + 1)
        W[i - start_idx + 1, i] = 1.0
    end
    return W
end

W1 = create_filter_matrix(1, 2, 4)
W2 = create_filter_matrix(3, 4, 4)

1
2
1
2


2×4 Matrix{Float64}:
 0.0  0.0  1.0  0.0
 0.0  0.0  0.0  1.0

In [ ]:
function create_expand_matrix(start_idx, end_idx, expand_dim)
    input_dim = end_idx - start_idx + 1

    W = zeros(expand_dim, input_dim)
    for i in start_idx:end_idx
        # println(i - start_idx + 1)
        W[i, i - start_idx + 1] = 1.0
    end
    return W
end

expand_m = create_expand_matrix(1, 2, 4)

[ Info: Precompiling Flux [587475ba-b771-5e3f-ad9e-33799f191a9c]


1
2


LoadError: MethodError: no method matching create_bias(::Matrix{Float64}, ::Float64, ::Int64)

[0mClosest candidates are:
[0m  create_bias(::AbstractArray, [91m::Bool[39m, ::Integer...)
[0m[90m   @[39m [36mFlux[39m [90m~/.julia/packages/Flux/n3cOc/src/[39m[90m[4mutils.jl:516[24m[39m
[0m  create_bias(::AbstractArray, [91m::AbstractArray[39m, ::Integer...)
[0m[90m   @[39m [36mFlux[39m [90m~/.julia/packages/Flux/n3cOc/src/[39m[90m[4mutils.jl:519[24m[39m
